In [ ]:
import pandas as pd

In [ ]:
ratings_df = pd.read_csv(f'datasets/netflix/ratings.csv.gz')
ratings_df

In [ ]:
movies_df = pd.read_csv(f'datasets/netflix/movies.csv.gz')
movies_df

In [ ]:
num_of_users = ratings_df['user_id'].nunique()
num_of_movies = ratings_df['movie_id'].nunique()
num_of_ratings = ratings_df.shape[0]
ratings = ratings_df['rating'].unique()
ratings.sort()
print(f'# users: {num_of_users}')
print(f'# movies: {num_of_movies}')
print(f'# ratings: {num_of_ratings}')
print(ratings)

In [ ]:
users_ratings = ratings_df.groupby('user_id').agg(
    rating_count=('rating', 'count'),
    rating_mean=('rating', 'mean'),
    # rating_std=('rating', 'std'),
    rating_var=('rating', 'var')
)
users_ratings.reset_index(inplace=True)
users_ratings.head()


In [ ]:
sorted = users_ratings.sort_values(by='rating_count')
sorted['cumulative_rating_count'] = sorted['rating_count'].cumsum()
sorted.reset_index(inplace=True)
sorted
sorted.plot.line(y='cumulative_rating_count')

In [ ]:
# group_by_num_ratings['total_ratings'] = group_by_num_ratings['count'] * group_by_num_ratings['mean']
# variances and means are uncorelated, and the users that we aggregate have the same amount of ratings
# therefore we can aggregate their mean and variance
group_by_num_ratings = users_ratings.groupby('rating_count').agg(
    group_size=('rating_count', 'size'),
    group_rating_mean=('rating_mean', 'mean'),
    group_rating_var=('rating_var', 'mean')
)
group_by_num_ratings.reset_index(inplace=True)
group_by_num_ratings['total_ratings'] = group_by_num_ratings['group_size'] * group_by_num_ratings['rating_count']
group_by_num_ratings
# group_by_num_ratings.head()
# user_ratings_groups = pd.cut(users_ratings['count'], bins=10)
# user_ratings_groups.head()

In [ ]:
# group_by_num_ratings.describe()
limit_to_max_1000_ratings = group_by_num_ratings.where(group_by_num_ratings['rating_count'] < 10000)
limit_to_max_1000_ratings['cumulative_total_ratings'] = limit_to_max_1000_ratings['total_ratings'].cumsum()
limit_to_max_1000_ratings['cumulative_group_size'] = limit_to_max_1000_ratings['group_size'].cumsum()
# limit_to_max_1000_ratings['cumulative_total_ratings_sqrt'] = limit_to_max_1000_ratings['cumulative_total_ratings']
# group_by_num_ratings['cumulative_total_ratings']
limit_to_max_1000_ratings.plot.line(x='rating_count', y =['total_ratings'])

In [ ]:
ratings_df.head()
users_ratings = ratings_df.groupby('userId').agg({'rating': ['count', 'mean']})
group_by_num_of_ratings =
users_ratings.head()


# unique, counts = np.unique(users_ratings, return_counts=True)
# df = pd.DataFrame({'how_many_rated': unique, 'user_count': counts})
# df['total_ratings'] = df['how_many_rated'] * df['user_count']
# df.head()
# ratings_distribution = [(u_item, u_item * c_item) for u_item, c_item in zip(unique, counts) if c_item < 10000]
# ratings_distribution = pd.DataFrame(ratings_distribution, columns=['ratedCount', 'totalRatingsForGroup'])

# ratings_distribution.plot.scatter(x='ratedCount', y='totalRatingsForGroup')


# num_with_ratings = pd.DataFrame(zip(unique, counts), columns=['ratedCount', 'numUsers'])
# num_with_ratings.head()
# num_with_ratings.plot(x='ratedCount', y='numUsers')

# num_users_with_num_ratings = users_ratings.groupby().size()
# num_users_with_num_ratings.sort()
# num_users_with_num_ratings